In [1]:
from typing import Tuple
import numpy as np
import deepgp
import matplotlib.pyplot as plt

import GPy
from GPy.models import GPRegression
from emukit.test_functions import forrester_function
from emukit.core.initial_designs import RandomDesign
from emukit.model_wrappers import GPyModelWrapper
from emukit.bayesian_optimization.acquisitions import ExpectedImprovement, NegativeLowerConfidenceBound, ProbabilityOfImprovement
from emukit.core.optimization import GradientAcquisitionOptimizer
from emukit.sensitivity.monte_carlo import MonteCarloSensitivity
from gpflow.kernels import RBF, White, Linear
from tqdm import tqdm

In [18]:
from simulator import MainSimulator, TinySimulator
from world import DebugInfo
from pprint import pprint

main_simulator = MainSimulator()

mutation_rates = {
    "size": 0,
    "speed": 1,
    "vision": 0,
    "aggression": 0
}

days_log = []
for i in tqdm(range(1)):
    main_simulator = MainSimulator()
    days_survived, log = main_simulator.run(mutation_rates, debug_info=DebugInfo(
        period=10, should_display_day=True, should_display_grid=False, should_display_traits=False), max_days=10000)
    days_log.append(days_survived)
    print(days_survived)


# for log_item in log:
#     print(log_item)
print(log[-1])

  0%|                                                                                                                                                                                                                                                                                                 | 0/1 [00:00<?, ?it/s]

Food generation likelihood: 1.4924141613183488
Food generation likelihood: 1.4871438652232216
Food generation likelihood: 1.480536940538561
Food generation likelihood: 1.4726271151301735
Food generation likelihood: 1.463452784586378
Food generation likelihood: 1.4530567041876021
Food generation likelihood: 1.4414856540816547
Food generation likelihood: 1.4287900816800168
Food generation likelihood: 1.4150237254544173
Food generation likelihood: 1.4002432244099798
Day number: 10
Population: 5082
Food generation likelihood: 1.3845077175420295
Food generation likelihood: 1.367878437550251
Food generation likelihood: 1.3504183029894732
Food generation likelihood: 1.332191512885219
Food generation likelihood: 1.3132631476394918
Food generation likelihood: 1.2936987798040507
Food generation likelihood: 1.273564098011204
Food generation likelihood: 1.2529245470328825
Food generation likelihood: 1.2318449865946015
Food generation likelihood: 1.2103893712091416
Day number: 20
Population: 4112
F

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:16<00:00, 16.01s/it]

Food generation likelihood: 0.5173826949312774
Food generation likelihood: 0.5268147482648545
Food generation likelihood: 0.5367102087090513
Food generation likelihood: 0.5470753380377172
Food generation likelihood: 0.5579161197505161
Day number: 840
Population: 3
Food generation likelihood: 0.5692381823285757
Food generation likelihood: 0.5810467175108818
Food generation likelihood: 0.59334639359262
Food generation likelihood: 0.6061412637969676
Food generation likelihood: 0.6194346698286505
Food generation likelihood: 0.6332291407809653
Food generation likelihood: 0.6475262876379346
Food generation likelihood: 0.6623266936897164
Food generation likelihood: 0.6776298012619445
Food generation likelihood: 0.6934337952482357
Day number: 850
Population: 2
850
LogItem(day=850, num_species_alive=0, temperature=22.42227528729705, probability_of_food=0.6934337952482357, traits_dict={'size': [5.0, 5.0, 5.0, 5.0], 'speed': [2.8039757390762237, 2.7696982885553516, 2.671961333705235, 2.5622960475

In [ ]:
class SingleTraitEmulator:
    def __init__(self, X, Y, kernel, noise=1e-10):
        self.model = GPyModelWrapper(GPRegression(X, Y, kernel, noise))
    
    def predict(self, X) -> Tuple[np.ndarray, np.ndarray]:
        return self.model.predict(X)
    
    def set_data(self, X: np.ndarray, Y: np.ndarray) -> None:
        self.model.set_data(X, Y)

#TODO: Data Preprocessing
X_size, Y_size = np.array([0,1,2,3]), np.array([1,3,7,2])
# Assuming a general quadratic relationship between mutation rate and survival years
size_kernel = GPy.kern.RatQuad(input_dim=1, power=1e4) * GPy.kern.Linear(1) * GPy.kern.Linear(-1)
size_emulator = SingleTraitEmulator(X_size, Y_size, size_kernel)

X_speed, Y_speed = np.array([0,1,2,3]), np.array([1,3,7,2])
speed_kernel = GPy.kern.RatQuad(input_dim=1, power=1e4) * GPy.kern.Linear(1) * GPy.kern.Linear(-1)
speed_emulator = SingleTraitEmulator(X_speed, Y_speed, speed_kernel)

X_vision, Y_vision = np.array([0,1,2,3]), np.array([1,3,7,2])
vision_kernel = GPy.kern.RatQuad(input_dim=1, power=1e4) * GPy.kern.Linear(1) * GPy.kern.Linear(-1)
vision_emulator = SingleTraitEmulator(X_vision, Y_vision, vision_kernel)

X_aggression, Y_agression = np.array([0,1,2,3]), np.array([1,3,7,2])
aggression_kernel = GPy.kern.RatQuad(input_dim=1, power=1e4) * GPy.kern.Linear(1) * GPy.kern.Linear(-1)
aggression_emulator = SingleTraitEmulator(X_aggression, Y_agression, aggression_kernel)

In [ ]:
def plot_prediction(X,Y,x_plot,mu_plot,var_plot,axis):
    axis.plot(X, Y, "ro", markersize=10, label="Observations")
    axis.plot(x_plot[:, 0], mu_plot[:, 0], "C0", label="Model")
    axis.fill_between(x_plot[:, 0],
                     mu_plot[:, 0] + np.sqrt(var_plot)[:, 0],
                     mu_plot[:, 0] - np.sqrt(var_plot)[:, 0], color="C0", alpha=0.6)
    axis.fill_between(x_plot[:, 0],
                     mu_plot[:, 0] + 2 * np.sqrt(var_plot)[:, 0],
                     mu_plot[:, 0] - 2 * np.sqrt(var_plot)[:, 0], color="C0", alpha=0.4)
    axis.fill_between(x_plot[:, 0],
                     mu_plot[:, 0] + 3 * np.sqrt(var_plot)[:, 0],
                     mu_plot[:, 0] - 3 * np.sqrt(var_plot)[:, 0], color="C0", alpha=0.2)
    axis.legend(loc=2, prop={'size': 10})
    axis.set(xlabel=r"$x$", ylabel=r"$f(x)$")
    axis.grid(True)

In [ ]:
def plot_acquisition_functions(x_plot, ei_plot, nlcb_plot, pi_plot, x_new, axis):
    axis.plot(x_plot, (ei_plot - np.min(ei_plot)) / (np.max(ei_plot) - np.min(ei_plot)), "green", label="EI")
    axis.plot(x_plot, (nlcb_plot - np.min(nlcb_plot)) / (np.max(nlcb_plot) - np.min(nlcb_plot)), "purple", label="NLCB")
    axis.plot(x_plot, (pi_plot - np.min(pi_plot)) / (np.max(pi_plot) - np.min(pi_plot)), "darkorange", label="PI")
    
    axis.axvline(x_new, color="red", label="x_next", linestyle="--")
    axis.legend(loc=1, prop={'size': 10})
    axis.set(xlabel=r"$x$", ylabel=r"$f(x)$")
    axis.grid(True)

In [ ]:
x_plot = np.linspace(0, 10, 1000)[:, None]
X = np.array([[0],[5], [10]])
Y = np.array([[0]])
for x in X:
    Y = np.append(Y_init,target_speed_function(x),axis=0)
Y = Y[1:]

ei_acquisition = ExpectedImprovement(emukit_speed_model)
nlcb_acquisition = NegativeLowerConfidenceBound(emukit_speed_model)
pi_acquisition = ProbabilityOfImprovement(emukit_speed_model)

In [ ]:
speed_model = GPRegression(X, Y, GPy.kern.RBF(1, lengthscale=1, variance=100), noise_var=1)
emukit_speed_model = GPyModelWrapper(speed_model)
mu_plot, var_plot = emukit_speed_model.predict(x_plot)
plot_prediction(X,Y,x_plot,mu_plot,var_plot,plt)
plt.show()

In [ ]:
iterations = 20
figure, axis = plt.subplots(iterations, 2, figsize=(10, iterations*3))

for i in tqdm(range(iterations)):
    mu_plot, var_plot = emukit_speed_model.predict(x_plot)
    plot_prediction(X,Y,x_plot,mu_plot,var_plot,axis[i,0])
    
    ei_plot = ei_acquisition.evaluate(x_plot)
    nlcb_plot = nlcb_acquisition.evaluate(x_plot)
    pi_plot = pi_acquisition.evaluate(x_plot)
    
    optimizer = GradientAcquisitionOptimizer(ParameterSpace([ContinuousParameter('x1', 0, 10)]))
    x_new, _ = optimizer.optimize(nlcb_acquisition)
    print("Next position to query:", x_new)
    plot_acquisition_functions(x_plot, ei_plot, nlcb_plot, pi_plot, x_new, axis[i,1])
    
    y_new = target_speed_function(x_new)
    X = np.append(X, x_new, axis=0)
    Y = np.append(Y, y_new, axis=0)
    emukit_speed_model.set_data(X, Y)

plt.show()

In [4]:
X_train = np.array([np.array([110,200,200,400]),np.array([300,252,300,400])])
Y_train = np.array([[100],[200]])

In [13]:
# DGP using deepgp library
Q = 5
num_layers = 1
kern1 = GPy.kern.RBF(Q,ARD=True) + GPy.kern.Bias(Q)
kern2 = GPy.kern.RBF(X_train.shape[1],ARD=True) + GPy.kern.Bias(X_train.shape[1])
num_inducing = 4 # Number of inducing points to use for sparsification
back_constraint = False # Whether to use back-constraint for variational posterior
encoder_dims=[[300],[150]] # Dimensions of the MLP back-constraint if set to true

dgp_model = deepgp.DeepGP([X_train.shape[1], num_layers, Y_train.shape[1]], X_train, Y_train, kernels=[kern2,None], num_inducing=num_inducing, back_constraint=back_constraint, encoder_dims=encoder_dims)

for i in range(len(dgp_model.layers)):
    output_var = dgp_model.layers[i].Y.var() if i==0 else dgp_model.layers[i].Y.mean.var()
    dgp_model.layers[i].Gaussian_noise.variance = output_var*0.01
    dgp_model.layers[i].Gaussian_noise.variance.fix()

dgp_model.optimize(max_iters=800, messages=True)
for i in range(len(dgp_model.layers)):
    dgp_model.layers[i].Gaussian_noise.variance.unfix()
dgp_model.optimize(max_iters=1200, messages=True)

  latent_space.  |   value  |  constraints  |  priors
  mean           |  (2, 1)  |               |        
  variance       |  (2, 1)  |      +ve      |         1 1
[[110 200 200 400]
 [300 252 300 400]] 4 4


In [14]:
display(dgp_model)

deepgp.,value,constraints,priors
obslayer.inducing inputs,"(4, 1)",,
obslayer.sum.rbf.variance,9108.78661789169,+ve,
obslayer.sum.rbf.lengthscale,127.98052888600307,+ve,
obslayer.sum.bias.variance,11327.789851930656,+ve,
obslayer.Gaussian_noise.variance,25.0,+ve,
obslayer.Kuu_var,"(4,)",+ve,
obslayer.latent space.mean,"(2, 1)",,
obslayer.latent space.variance,"(2, 1)",+ve,
layer_1.inducing inputs,"(4, 4)",,
layer_1.rbf.variance,4139.84455596258,+ve,


In [25]:
x_plot = np.linspace(0, 1000, 1000)[:, None]
x_new = np.stack((x_plot,x_plot,x_plot,x_plot),axis = -1)
Y_pred = dgp_model.predict(np.array([[10,10,10,10]]))

In [ ]:
def target_size_function(x):
    mutation_rates = {
        "size": x,
        "speed": 0,
        "vision": 0,
        "aggression": 0
    }
    days_survived, log = main_simulator.run(mutation_rates, debug_info=DebugInfo(
        period=10, should_display_day=True, should_display_grid=False, should_display_traits=False), max_days=10000)
    return days_survived
    
def target_speed_function(x):
    mutation_rates = {
        "size": 0,
        "speed": x,
        "vision": 0,
        "aggression": 0
    }
    days_survived, log = main_simulator.run(mutation_rates, debug_info=DebugInfo(
        period=10, should_display_day=True, should_display_grid=False, should_display_traits=False), max_days=10000)
    return days_survived

def target_vision_function(x):
    mutation_rates = {
        "size": 0,
        "speed": 0,
        "vision": x,
        "aggression": 0
    }
    days_survived, log = main_simulator.run(mutation_rates, debug_info=DebugInfo(
        period=10, should_display_day=True, should_display_grid=False, should_display_traits=False), max_days=10000)
    return days_survived

def target_aggression_function(x):
    mutation_rates = {
        "size": 0,
        "speed": 0,
        "vision": 0,
        "aggression": x
    }
    days_survived, log = main_simulator.run(mutation_rates, debug_info=DebugInfo(
        period=10, should_display_day=True, should_display_grid=False, should_display_traits=False), max_days=10000)
    return days_survived

def target_function(X):
    mutation_rates = {
        "size": X[0],
        "speed": X[1],
        "vision": X[2],
        "aggression": X[3]
    }
    days_survived, log = main_simulator.run(mutation_rates, debug_info=DebugInfo(
        period=10, should_display_day=True, should_display_grid=False, should_display_traits=False), max_days=10000)
    return days_survived

In [30]:
x_plot = np.linspace(0, 20, 1000)[:, None]

X_size = np.array([[0],[1], [20]])
X_speed = np.array([[0],[1], [20]])
X_vision = np.array([[0],[1], [20]])
X_aggression = np.array([[0],[1], [20]])

Y_size = np.array([[0]])
Y_speed = np.array([[0]])
Y_vision = np.array([[0]])
Y_aggression = np.array([[0]])

In [ ]:
for x in X_size:
    Y_size = np.append(Y_size,target_size_function(x),axis=0)
    
Y_size = Y_size[1:]

In [ ]:
for x in X_speed:
    Y_speed = np.append(Y_speed,target_speed_function(x),axis=0)

Y_speed = Y_speed[1:]

In [ ]:
for x in X_vision:
    Y_vision = np.append(Y_vision,target_vision_function(x),axis=0)
    
Y_vision = Y_vision[1:]

In [ ]:
for x in X_aggression:
    Y_aggression = np.append(Y_aggression,target_aggression_function(x),axis=0)

Y_aggression = Y_agression[1:]

In [ ]:
size_model = GPRegression(X, Y, GPy.kern.RBF(1, lengthscale=1, variance=100), noise_var=1)
speed_model = GPRegression(X, Y, GPy.kern.RBF(1, lengthscale=1, variance=100), noise_var=1)
vision_model = GPRegression(X, Y, GPy.kern.RBF(1, lengthscale=1, variance=100), noise_var=1)
aggression_model = GPRegression(X, Y, GPy.kern.RBF(1, lengthscale=1, variance=100), noise_var=1)

emukit_size_model = GPyModelWrapper(size_model)
emukit_speed_model = GPyModelWrapper(speed_model)
emukit_vision_model = GPyModelWrapper(vision_model)
emukit_aggression_model = GPyModelWrapper(agression_model)

size_ei_acquisition = ExpectedImprovement(emukit_size_model)
size_nlcb_acquisition = NegativeLowerConfidenceBound(emukit_size_model)
size_pi_acquisition = ProbabilityOfImprovement(emukit_size_model)

speed_ei_acquisition = ExpectedImprovement(emukit_speed_model)
speed_nlcb_acquisition = NegativeLowerConfidenceBound(emukit_speed_model)
speed_pi_acquisition = ProbabilityOfImprovement(emukit_speed_model)

vision_ei_acquisition = ExpectedImprovement(emukit_vision_model)
vision_nlcb_acquisition = NegativeLowerConfidenceBound(emukit_vision_model)
vision_pi_acquisition = ProbabilityOfImprovement(emukit_vision_model)

aggression_ei_acquisition = ExpectedImprovement(emukit_aggression_model)
aggression_nlcb_acquisition = NegativeLowerConfidenceBound(emukit_aggression_model)
aggression_pi_acquisition = ProbabilityOfImprovement(emukit_aggression_model)

In [ ]:
X_train = np.array([[1,1,1,1, Y_size[1], Y_speed[1], Y_vision[1], Y_aggression[1]],
                   [1,0,0,0, Y_size[1], Y_speed[0], Y_vision[0], Y_aggression[0]],
                   [1,1,1,1, Y_size[1], Y_speed[1], Y_vision[1], Y_aggression[1]],
                   [1,1,1,1, Y_size[1], Y_speed[1], Y_vision[1], Y_aggression[1]],
                   [1,1,1,1, Y_size[1], Y_speed[1], Y_vision[1], Y_aggression[1]]])
Y_train = np.array([[target_function([1,1,1,1])]])
Q = 5
num_layers = 1
kern1 = GPy.kern.RBF(Q,ARD=True) + GPy.kern.Bias(Q)
kern2 = GPy.kern.RBF(X_train.shape[1],ARD=True) + GPy.kern.Bias(X_train.shape[1])
num_inducing = 4 # Number of inducing points to use for sparsification
back_constraint = False # Whether to use back-constraint for variational posterior
encoder_dims=[[300],[150]] # Dimensions of the MLP back-constraint if set to true

In [31]:
def upper_confidence_bound(y_pred, y_std, beta):
    ucb = y_pred + beta * y_std
    return ucb

beta = 2.0

In [ ]:
iterations = 20
figure, axis = plt.subplots(iterations, 2, figsize=(10, iterations*3))

for i in tqdm(range(iterations)):
    mu_speed_plot, var_speed_plot = emukit_speed_model.predict(x_plot)
    ei_speed_plot = speed_ei_acquisition.evaluate(x_plot)
    nlcb_speed_plot = speed_nlcb_acquisition.evaluate(x_plot)
    pi_speed_plot = speed_pi_acquisition.evaluate(x_plot)
    
    size_optimizer = GradientAcquisitionOptimizer(ParameterSpace([ContinuousParameter('x1', 0, 20)]))
    x_size_new, _ = size_optimizer.optimize(size_nlcb_acquisition)
    speed_optimizer = GradientAcquisitionOptimizer(ParameterSpace([ContinuousParameter('x1', 0, 20)]))
    x_speed_new, _ = speed_optimizer.optimize(speed_nlcb_acquisition)
    vision_optimizer = GradientAcquisitionOptimizer(ParameterSpace([ContinuousParameter('x1', 0, 20)]))
    x_vision_new, _ = vision_optimizer.optimize(vision_nlcb_acquisition)
    aggression_optimizer = GradientAcquisitionOptimizer(ParameterSpace([ContinuousParameter('x1', 0, 20)]))
    x_aggression_new, _ = aggression_optimizer.optimize(agression_nlcb_acquisition)

    print("Next position to query:", x_size_new, x_speed_new, x_vision_new, x_agression_new)
    
    y_new = target_speed_function(x_new)
    X = np.append(X, x_new, axis=0)
    Y = np.append(Y, y_new, axis=0)
    emukit_speed_model.set_data(X, Y)

plt.show()

In [ ]:
x